In [ ]:
import torch
from torch import nn
import numpy as np

In [ ]:
text = ['hey how are you', 'good i am fine', 'good bye', 'have a nice day']

chars = set(''.join(text))

int2char = dict(enumerate(chars))

char2int = {char: ind for ind, char in int2char.items()}

In [ ]:
print(char2int)

{'b': 0, 'f': 1, 'y': 2, 'w': 3, 'v': 4, 'd': 5, 'r': 6, ' ': 7, 'o': 8, 'h': 9, 'n': 10, 'g': 11, 'a': 12, 'u': 13, 'e': 14, 'm': 15, 'c': 16, 'i': 17}


In [ ]:
maxlen = len(max(text, key=len))
print(maxlen)

15


In [ ]:
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

In [ ]:
input_seq = []
target_seq = []

for i in range(len(text)):
  input_seq.append(text[i][:-1])
  target_seq.append(text[i][1:])

  print("Input Seq : {}  \n Target Seq : {}".format(input_seq[i], target_seq[i]))

Input Seq : hey how are yo  
 Target Seq : ey how are you
Input Seq : good i am fine  
 Target Seq : ood i am fine 
Input Seq : good bye        
 Target Seq : ood bye       
Input Seq : have a nice da  
 Target Seq : ave a nice day


In [ ]:
for i in range(len(text)):
  input_seq[i] = [char2int[character] for character in input_seq[i]]
  target_seq[i] = [char2int[character] for character in target_seq[i]]

In [ ]:
dict_size = len(char2int)
seq_len = maxlen-1
batch_size = len(text)

def one_hot_encoder(sequence, dict_size, seq_len, batch_size):
  feature = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
  for i in range(batch_size):
    for u in range(seq_len):
      feature[i, u, sequence[i][u]]=1
  return feature


In [ ]:
input_seq = one_hot_encoder(input_seq, dict_size, seq_len, batch_size)

In [ ]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [ ]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
class Model(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)
  def forward(self, x):
        batch_size = x.size(0)

        hidden = self.init_hidden(batch_size)

        out, hidden = self.rnn(x, hidden)

        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

  def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

In [ ]:
model= Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model = model.to(device)

n_epochs = 70
lr = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [ ]:
n_epochs=30
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs+1):
  optimizer.zero_grad()
  output, hidden = model(input_seq)
  output = output.to(device)
  target_seq = target_seq.to(device)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward()
  optimizer.step()

  if epoch%10  == 0:
    print('Epoch: {}/{}......'.format(epoch, n_epochs), end=' ')
    print("Loss : {:.4f}".format(loss.item()))



Epoch: 10/30...... Loss : 0.2516
Epoch: 20/30...... Loss : 0.2033
Epoch: 30/30...... Loss : 0.1684


In [ ]:
def predict(model, character):
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encoder(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)

    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [ ]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
sample(model, 15, 'have')

'have a nice day'